# **🧠 O que é o ciclo BDI?**

> ⚠️ Este Colab não foi feito para execução do codigo, criei por motivos de explicabilidade do codigo. O codigo deve ser rodado localmente no caminho /code.

O **BDI (Belief-Desire-Intention)** é uma arquitetura de agentes baseada na forma como humanos tomam decisões racionais.

O ciclo BDI funciona em três camadas:

*   **Beliefs (Crenças): o que o agente sabe sobre o mundo.**
*   **Desires (Desejos): o que ele quer alcançar.**
*   **Intentions (Intenções): os planos escolhidos para agir com base nas crenças e desejos.**


O agente repete continuamente esse ciclo:

1. **Atualiza suas crenças com novas informações (ex: uma erva daninha detectada).**

2. **Seleciona desejos relevantes com base nesse estado.**

3. **Escolhe planos válidos (cujos contextos estão de acordo com as crenças).**

4. **Executa ações para alcançar os objetivos.**

5. **Volta ao início e reavalia tudo.**










In [ ]:
import time
import threading
import requests
from flask import Flask, request, jsonify

### **Agente Herbicida** — `agent_herbicida.py`
- O agente herbicida simula um drone que pulveriza o local da plantação que foi detectado como infestado por ervas daninhas. Então espera as crenças enviadas do supervisor para agir:
  - `aplicar_herbicida = True`
  - `erva_daninha_detectada = True`
  - `coordenadas_agente_campo = True`
- Após executar a ação, atualiza suas crenças e notifica os agentes de campo e supervisor que terminou sua missão.

> A ***PlanLibrary*** armazena os **planos do agente**, associando cada **objetivo (goal)** a um **contexto necessário (pré-condições)** e a uma **sequência de açõe**s que devem ser executadas quando aquele objetivo for ativado.
A função ***get_plan()*** verifica se o contexto de um plano é compatível com **as crenças atuais (beliefs)**. Se for, retorna o plano associado; caso contrário, ignora.

In [ ]:


class PlanLibrary:
    def __init__(self):
        self.plans = {}

    def set_plan_library(self, planlb):
        self.plans = planlb

    def add_plan(self, goal, prec, plan):
        self.plans[goal] = {'context': prec, 'plan': plan}

    def get_plan(self, goal, bb):
        if goal in self.plans:
            for key, val in self.plans[goal]['context'].items():
                if key not in bb or bb[key] != val:
                    return None
            return self.plans[goal]['plan']
        return None



## 🔍 `class Action`

A classe `Action` do agente herbicida define as tarefas executadas quando o plano de contenção é ativado. Ela representa o comportamento simulado de um drone ou robô que se desloca até o local afetado e executa a missão de eliminação de ervas daninhas.



### 🚜 `ir_ate_agente_campo()`
- Simula o deslocamento até a área contaminada.
- Após um tempo de espera (simulando a viagem), considera que a missão foi executada com sucesso.
- Atualiza suas crenças para indicar:
  - Que não há mais ervas daninhas.
  - Que a missão foi concluída.
  - Que o herbicida não precisa mais ser aplicado.

**Fluxo:**
1. A missão é ativada com base nas crenças enviadas pelo supervisor.
2. O herbicida vai até o local e realiza a contenção.
3. Atualiza suas crenças internas para refletir o novo estado do ambiente.
4. Envia confirmações para os agentes envolvidos.


In [ ]:
class Action:
    def ir_ate_agente_campo(self):
        print("🚜 Indo até o local do agente de campo...")
        time.sleep(5)
        print("✅ Chegamos no local e eliminamos as ervas daninhas.")
        agente.add_beliefs({"erva_daninha_detectada": False})
        agente.add_beliefs({"aplicar_herbicida": False})
        print("✅ Voltando para base.")
        agente.add_beliefs({"missao_concluida": True})

    def informar_agente_campo(self):
        print("📡 Enviando confirmação para o agente de campo...")
        try:
            crença_agente_campo = {"erva_daninha": False}
            res = requests.post("http://127.0.0.1:5002/agent", json=crença_agente_campo)
            print(f"📬 Confirmação enviada | Status: {res.status_code}")
        except Exception as e:
            print(f"❌ Erro ao informar agente de campo: {e}")

        agente.add_beliefs({"contenção_comunicada": True})

    def informar_agente_supervisor(self):
        print("📡 Enviando confirmação para o agente de campo...")
        try:
            crença_agente_supervisor = {"erva_daninha": False}
            res = requests.post("http://127.0.0.1:5003/agent", json=crença_agente_supervisor)
            print(f"📬 Confirmação enviada | Status: {res.status_code}")
        except Exception as e:
            print(f"❌ Erro ao informar agente de campo: {e}")

        agente.add_beliefs({"contenção_comunicada": True})



### **🤖 Classe Agent (BDI)**

A classe `Agent` implementa a lógica principal de um agente inteligente baseado no modelo BDI (*Beliefs-Desires-Intentions*).

In [ ]:
class Agent:
    def __init__(self):
        self.beliefs = {}
        self.desires = []
        self.intention = []
        self.plan_library = PlanLibrary()

    def get_desires(self):
        return self.desires.pop() if self.desires else None

    def add_desires(self, desire):
        self.desires.append(desire)

    def add_beliefs(self, beliefs):
        self.beliefs.update(beliefs)

    def set_plan_library(self, pl):
        self.plan_library.set_plan_library(pl)

    def update_intention(self, goal):
        plan = self.plan_library.get_plan(goal, self.beliefs)
        if plan:
            self.intention.extend(plan)

    def execute_intention(self):
        while self.intention:
            next_goal = self.intention.pop()
            if self.plan_library.get_plan(next_goal, self.beliefs) is None:
                action = getattr(Action(), next_goal)
                action()
            else:
                self.intention.extend(self.plan_library.get_plan(next_goal, self.beliefs))



### 🗺️ **Planos Definidos (set_plan_library)**

- **iniciar_missao:**  
  Ativado quando todas as condições críticas são verdadeiras (missão confirmada, localização recebida, e erva daninha detectada). O agente simula o deslocamento até o local e realiza a contenção.

- **confirmar_limpeza:**  
  Após a missão, se não houver mais erva daninha, e se ainda não houver comunicação registrada, o agente envia notificações para o agente de campo e o supervisor informando que a missão foi concluída.

🌐 Endpoint `/agent`

- **GET:** permite consultar as crenças atuais do agente herbicida.
- **POST:** recebe crenças enviadas por outros agentes (como o supervisor), atualizando seu estado interno.

Esse endpoint permite que o agente herbicida se mantenha sincronizado com o restante do sistema e reaja automaticamente ao fluxo de decisões do ecossistema BDI.

In [ ]:
agente = Agent()

# Crenças iniciais
agente.add_beliefs({
    "aplicar_herbicida": False,
    "coordenadas_agente_campo": False,
    "erva_daninha_detectada": False,
    "contenção_comunicada": False,
    "missao_concluida": False
})

# Biblioteca de planos
agente.set_plan_library({
    "iniciar_missao": {
        "context": {
            "aplicar_herbicida": True,
            "coordenadas_agente_campo": True,
            "erva_daninha_detectada": True
        },
        "plan": ["ir_ate_agente_campo"]
    },
    "confirmar_limpeza": {
        "context": {
            "erva_daninha_detectada": False, "contenção_comunicada": False, "missao_concluida": True
        },
        "plan": ["informar_agente_campo", "informar_agente_supervisor"]
    }
})

app = Flask(__name__)

@app.route("/agent", methods=["GET", "POST"])
def endpoint_agent():
    if request.method == "GET":
        return jsonify(agente.beliefs)
    if request.method == "POST":
        novas = request.json
        agente.add_beliefs(novas)
        print(f"🧠 Novas crenças recebidas: {novas}")
        return jsonify({"msg": "Crenças atualizadas", "crencas": agente.beliefs})




### **🔁 Ciclo BDI Contínuo (ciclo_agente)**

Este é o **loop principal** que mantém o agente ativo e deliberando de forma autônoma.

In [ ]:
def ciclo_agente():
    while True:
        agente.add_desires("iniciar_missao")
        agente.add_desires("confirmar_limpeza")

        while True:
            goal = agente.get_desires()
            if not goal:
                break
            agente.update_intention(goal)
            agente.execute_intention()

        time.sleep(3)

if __name__ == "__main__":
    threading.Thread(target=ciclo_agente, daemon=True).start()
    print("🌱 Agente Herbicida rodando na porta 5004...")
    app.run(host="0.0.0.0", port=5004)